# Redis Basics Tutorial

Introduction aux bases de Redis avec notre dataset de films.

## 1. Connexion à Redis

In [ ]:
import sys
sys.path.append('/home/jovyan/work')

from pprint import pprint

import redis

# Connexion à Redis via un client Redis
r = redis.Redis(host='redis', port=6379, db=0)

# Tester la connexion
print("Connection test:", r.ping())

Connection test: True


## 2. Exploration globale de la db

In [10]:
num_keys = r.dbsize()
print(f"Nombre total de clés dans Redis: {num_keys}")

Nombre total de clés dans Redis: 8354


Comme la base de donnée contient un nombre raisonnable de clés, on peut récupérer toutes les clés et les parcourir rapidement.

In [ ]:
# Récupérer le type d'objet dans notre base de données

keys = r.keys('*')  # Récupérer toutes les clés
prefixes = set(key.decode('utf-8').split(':')[0] for key in keys)
print("Types d'objets dans Redis:", prefixes)

Types d'objets dans Redis: {'theater', 'user', 'actor', 'movie'}


In [ ]:
# Récupérer le nombre d'objets par catégorie

category_counts = {prefix: 0 for prefix in prefixes}

for key in keys:
    category = key.decode('utf-8').split(':')[0]
    category_counts[category] += 1

print("Nombre d'objets par catégorie:", category_counts)

Nombre d'objets par catégorie: {'theater': 117, 'user': 5996, 'actor': 1319, 'movie': 922}


Vérifier le type des clés

In [ ]:
# Récupérer les 10 premières clés

r.keys('*')[:10]

[b'user:1421',
 b'user:137',
 b'user:1282',
 b'user:5183',
 b'user:4084',
 b'user:919',
 b'user:2539',
 b'user:1014',
 b'user:4418',
 b'movie:863']

In [ ]:
# Pour connaître le type des clés dans la base de données 

key_type = r.type('user:1421')  # Vérifie le type de la clé

Type of 'user:1421': b'hash'
Type of 'movie:863': b'hash'


## 3. Récupérer des données

### Selon le type de la clé on choisit la bonne commande 
Pour un Hash, utilisez HGET ou HGETALL.
Pour une Liste, utilisez LRANGE ou LPOP.
Pour un Set, utilisez SMEMBERS.
Pour un Sorted Set, utilisez ZRANGE. 

In [ ]:
# Vérifier si la clé existe

r.exists('movie:1') == 1  # Renvoie 1 si la clé existe, 0 sinon

In [ ]:
# Récupérer un champ spécifique d'un Hash

r.hget('movie:1', 'title')

b'New Title'

In [ ]:
# Récupérer plusieurs champs d'un Hash

r.hmget('movie:1', ['title', 'genre'])

[b'New Title', b'Action']

In [ ]:
# Récupérer tous les champs d'un Hash

r.hgetall('movie:1')

## 4. Ajouter et modifier des données

HSET permet de:
- ajouter une donnée (clé - valeur) à la db si la clé spécifiée n'existe pas
- modifier une donnée si la clé spécifiée existe

HSET renvoie 1 si une donnée a été créée, 0 si une donnée a été modifié

In [ ]:
# Vérifier si la clé existe
print(f"Key exists: {r.exists('movie:1') == 1}")

# Récupérer un champ spécifique d'un Hash
print(f"Initial title: {r.hget('movie:1', 'title')})")

# Modifier un champ spécifique d'un Hash
r.hset('movie:1', 'title', 'Toy Story')

print(f"Title after modification: {r.hget('movie:1', 'title')}")

In [ ]:
# Vérifier si la clé existe
print(f"Key exists: {r.exists('movie:6000') == 1}")

# Ajouter une donnée à la db
r.hset('movie:6000', 'title', 'Inception')

r.hgetall('movie:6000')

In [ ]:
# HSET permet aussi de créer / modifier une clé avec plusieurs valeurs en une seule commande

r.hset('movie:6001', mapping={'title': 'Gladiator', 'genre': 'Action', 'year': 2000})

r.hgetall('movie:6001')

## 5. Supprimer des données